## Machine Reading Comprehension

### SQuAD 2.0 Data Sample

```json
{"version": "v2.0", "data": 
    [{"title": "Beyonc\u00e9", "paragraphs": 
        [{"qas": [{"question": "When did Beyonce start becoming popular?", "id": "56be85543aeaaa14008c9063", "answers": [{"text": "in the late 1990s", "answer_start": 269}], "is_impossible": false},
               {"question": "What areas did Beyonce compete in when she was growing up?", "id": "56be85543aeaaa14008c9065", "answers": [{"text": "singing and dancing", "answer_start": 207}], "is_impossible": false},
               {"question": "When did Beyonce leave Destiny's Child and become a solo singer?", "id": "56be85543aeaaa14008c9066", "answers": [{"text": "2003", "answer_start": 526}], "is_impossible": false},
               {"question": "In what city and state did Beyonce  grow up? ", "id": "56bf6b0f3aeaaa14008c9601", "answers": [{"text": "Houston, Texas", "answer_start": 166}], "is_impossible": false},
               {"question": "In which decade did Beyonce become famous?", "id": "56bf6b0f3aeaaa14008c9602", "answers": [{"text": "late 1990s", "answer_start": 276}], "is_impossible": false},
               {"question": "In what R&B group was she the lead singer?", "id": "56bf6b0f3aeaaa14008c9603", "answers": [{"text": "Destiny's Child", "answer_start": 320}], "is_impossible": false},
               {"question": "What album made her a worldwide known artist?", "id": "56bf6b0f3aeaaa14008c9604", "answers": [{"text": "Dangerously in Love", "answer_start": 505}], "is_impossible": false},
               {"question": "Who managed the Destiny's Child group?", "id": "56bf6b0f3aeaaa14008c9605", "answers": [{"text": "Mathew Knowles", "answer_start": 360}], "is_impossible": false},
               {"question": "When did Beyonc\u00e9 rise to fame?", "id": "56d43c5f2ccc5a1400d830a9", "answers": [{"text": "late 1990s", "answer_start": 276}], "is_impossible": false},
               {"question": "What role did Beyonc\u00e9 have in Destiny's Child?", "id": "56d43c5f2ccc5a1400d830aa", "answers": [{"text": "lead singer", "answer_start": 290}], "is_impossible": false},
               {"question": "What was the first album Beyonc\u00e9 released as a solo artist?", "id": "56d43c5f2ccc5a1400d830ab", "answers": [{"text": "Dangerously in Love", "answer_start": 505}], "is_impossible": false},
               {"question": "When did Beyonc\u00e9 release Dangerously in Love?", "id": "56d43c5f2ccc5a1400d830ac", "answers": [{"text": "2003", "answer_start": 526}], "is_impossible": false},
               {"question": "How many Grammy awards did Beyonc\u00e9 win for her first solo album?", "id": "56d43c5f2ccc5a1400d830ad", "answers": [{"text": "five", "answer_start": 590}], "is_impossible": false},
               {"question": "What was Beyonc\u00e9's role in Destiny's Child?", "id": "56d43ce42ccc5a1400d830b4", "answers": [{"text": "lead singer", "answer_start": 290}], "is_impossible": false},
               {"question": "What was the name of Beyonc\u00e9's first solo album?", "id": "56d43ce42ccc5a1400d830b5", "answers": [{"text": "Dangerously in Love", "answer_start": 505}], "is_impossible": false}],
               "context": "Beyonc\u00e9 Giselle Knowles-Carter (/bi\u02d0\u02c8j\u0252nse\u026a/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyonc\u00e9's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles \"Crazy in Love\" and \"Baby Boy\"."}```


In [1]:
!pip install bert4keras==0.7.7

  Created wheel for bert4keras: filename=bert4keras-0.7.7-cp36-none-any.whl size=36804 sha256=22c139dc942fea1d7ba677102f3d2ba1ed454f2740ced2594187bbf8e58ac69a
  Stored in directory: /root/.cache/pip/wheels/73/5f/2b/ee7932b73172503fdb429421c0b433f47a737c0d1c2a85fdd2
Successfully built bert4keras


In [2]:
import json, os
import numpy as np
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Layer, Dense, Permute
from keras.models import Model
from tqdm import tqdm

# 基本信息
maxlen = 512
epochs = 20
batch_size = 4
learing_rate = 2e-5

# 1) files path config

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
data_dir="/content/drive/My Drive/Colab Notebooks/Machine Reading Comprehension English/data"
output_dir='/content/drive/My Drive/Colab Notebooks/Machine Reading Comprehension English/data/output/'

##### Bert pre-train-models：https://github.com/google-research/bert#pre-trained-models

In [5]:
bert_dir = '/content/drive/My Drive/Colab Notebooks/Machine Reading Comprehension English/model/uncased_L-12_H-768_A-12'
config_path = f'{bert_dir}/bert_config.json'
checkpoint_path = f'{bert_dir}/bert_model.ckpt'
dict_path = f'{bert_dir}/vocab.txt'

In [6]:
os.listdir(bert_dir)

['vocab.txt',
 'bert_config.json',
 'bert_model.ckpt.index',
 'bert_model.ckpt.meta',
 'bert_model.ckpt.data-00000-of-00001']

In [7]:
os.listdir(data_dir)

['train-v2.0.json',
 'dev-v2.0.json',
 '__init__.py',
 '__pycache__',
 'output',
 'evaluate.py']

# 2) Data Loader: Read Data

In [8]:
def load_data(file_name):
    
    context_id_qas_list = []
    
    with open(file_name) as f:
        json_f = json.load(f)
    
    data = json_f['data']
    
    for paragraphs in data:
        context_qas_list = paragraphs['paragraphs']
        for context_qas in context_qas_list:
            
            context = context_qas['context']
            qas_list = context_qas['qas']
            
            for qas in qas_list:
                is_impossible = qas['is_impossible']
                if not is_impossible:
                    id_ = qas['id']
                    question = qas['question']
                    answer_dict = qas['answers']
                    answer_text = [item['text'] for item in answer_dict]
#                     answer_start = answer_dict['answer_start']
                    context_id_qas_list.append([id_,context,question,answer_text])
                                            
    return context_id_qas_list

In [9]:
train_data = load_data(data_dir+'/train-v2.0.json')

In [10]:
len(train_data)

86821

In [11]:
dev_data = train_data[500:]  ### only 500 for validation

In [12]:
train_data = train_data[:500] ### only 500 for training

In [13]:
train_data[:3]

[['56be85543aeaaa14008c9063',
  'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
  'When did Beyonce start becoming popular?',
  ['in the late 1990s']],
 ['56be85543aeaaa14008c9065',
  'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised

# 3) tokenizer: tokenize

In [14]:
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 4) sub-string search

In [15]:
def search(pattern, sequence):
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1

# 5) Data generator

In [16]:
class data_generator(DataGenerator):
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        
        for is_end, item in self.sample(random):
            
            context, question, answers = item[1:]
            
            token_ids, segment_ids = tokenizer.encode(
                question, context, max_length=maxlen
            )
            a = np.random.choice(answers)
            
            a_token_ids = tokenizer.encode(a)[0][1:-1]
            
            start_index = search(a_token_ids, token_ids)
            
            if start_index != -1:
                
                labels = [[start_index], [start_index + len(a_token_ids) - 1]]
                
                batch_token_ids.append(token_ids)
                
                batch_segment_ids.append(segment_ids)
                
                batch_labels.append(labels)
                
                if len(batch_token_ids) == self.batch_size or is_end:
                    
                    batch_token_ids = sequence_padding(batch_token_ids)
                    
                    batch_segment_ids = sequence_padding(batch_segment_ids)
                    
                    batch_labels = sequence_padding(batch_labels)
                    
                    yield [batch_token_ids, batch_segment_ids], batch_labels
                    
                    batch_token_ids, batch_segment_ids, batch_labels = [], [], []

In [17]:
train_generator = data_generator(train_data, batch_size)
# evaluator = Evaluator()

In [18]:
next(iter(train_generator))

([array([[  101,  2043,  2106, 20773,  2707,  3352,  2759,  1029,   102,
          20773, 21025, 19358, 22815,  1011,  5708,  1006,  1013, 12170,
          23432, 29715,  3501, 29678, 12325, 29685,  1013, 10506,  1011,
          10930,  2078,  1011,  2360,  1007,  1006,  2141,  2244,  1018,
           1010,  3261,  1007,  2003,  2019,  2137,  3220,  1010,  6009,
           1010,  2501,  3135,  1998,  3883,  1012,  2141,  1998,  2992,
           1999,  5395,  1010,  3146,  1010,  2016,  2864,  1999,  2536,
           4823,  1998,  5613,  6479,  2004,  1037,  2775,  1010,  1998,
           3123,  2000,  4476,  1999,  1996,  2397,  4134,  2004,  2599,
           3220,  1997,  1054,  1004,  1038,  2611,  1011,  2177, 10461,
           1005,  1055,  2775,  1012,  3266,  2011,  2014,  2269,  1010,
          25436, 22815,  1010,  1996,  2177,  2150,  2028,  1997,  1996,
           2088,  1005,  1055,  2190,  1011,  4855,  2611,  2967,  1997,
           2035,  2051,  1012,  2037, 14221,  2387,

# 6) Masked Softmax

In [19]:
class MaskedSoftmax(Layer):
    """
    softmax the last dimension, and mask the padding
    """
    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask, 2)
            inputs = inputs - (1.0 - mask) * 1e12
        return K.softmax(inputs, 1)

# 7) model construction

In [20]:
model = build_transformer_model(
    config_path,
    checkpoint_path,
)

output = Dense(2)(model.output)
output = MaskedSoftmax()(output)
output = Permute((2, 1))(output)

model = Model(model.input, output)
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
_______________________________________________________________________________________

# 8)  Evaluation function

In [21]:
def sparse_categorical_crossentropy(y_true, y_pred):
    
    y_true = K.reshape(y_true, K.shape(y_pred)[:-1])
    y_true = K.cast(y_true, 'int32')
    y_true = K.one_hot(y_true, K.shape(y_pred)[2])
    
    return K.mean(K.categorical_crossentropy(y_true, y_pred))


def sparse_accuracy(y_true, y_pred):
    
    y_true = K.reshape(y_true, K.shape(y_pred)[:-1])
    y_true = K.cast(y_true, 'int32')
    
    y_pred = K.cast(K.argmax(y_pred, axis=2), 'int32')
    return K.mean(K.cast(K.equal(y_true, y_pred), K.floatx()))

# 9) Compile model

In [22]:
model.compile(
    loss=sparse_categorical_crossentropy,
    optimizer=Adam(learing_rate),
    metrics=[sparse_accuracy]
)

# 10) Extract Answer Function

In [23]:
def extract_answer(question, context, max_a_len=16):
    max_q_len = 64
    q_token_ids = tokenizer.encode(question, max_length=max_q_len)[0]
    c_token_ids = tokenizer.encode(
        context, max_length=maxlen - len(q_token_ids) + 1
    )[0]
    token_ids = q_token_ids + c_token_ids[1:]
    segment_ids = [0] * len(q_token_ids) + [1] * (len(c_token_ids) - 1)
    c_tokens = tokenizer.tokenize(context)[1:-1]
    mapping = tokenizer.rematch(context, c_tokens)
    probas = model.predict([np.array([token_ids]), np.array([segment_ids])])[0]
    probas = probas[:, len(q_token_ids):-1]
    start_end, score = None, -1
    for start, p_start in enumerate(probas[0]):
        for end, p_end in enumerate(probas[1]):
            if end >= start and end < start + max_a_len:
                if p_start * p_end > score:
                    start_end = (start, end)
                    score = p_start * p_end
                    
    start, end = start_end
    return context[mapping[start][0]:mapping[end][-1] + 1]

# 11) Predicted Result Generation

In [24]:
def predict_to_file(infile, out_file):
    """write out the predict result
    """
    fw = open(out_file, 'w', encoding='utf-8')
    R = {}
    for d in tqdm(load_data(infile)[:500]): ### only 500
        a = extract_answer(d[2], d[1])
        R[d[0]] = a
    R = json.dumps(R, ensure_ascii=False, indent=4)
    fw.write(R)
    fw.close()

#12) Official Evaluation Function

In [25]:
import sys
import io
import json
sys.path.append(data_dir)
import evaluate as evaluate_utils #### official evaluation function
from collections import OrderedDict

In [26]:
def evaluate(pred_file,pred_result_file):

    predict_to_file(pred_file,pred_result_file)

    dataset_json = json.load(io.open(pred_file))
    dataset = dataset_json['data']
    preds = json.load(io.open(pred_result_file))

    exact_raw, f1_raw = evaluate_utils.get_raw_scores(dataset, preds)

    output_result = evaluate_utils.make_eval_dict(exact_raw, f1_raw)

    return output_result

In [27]:
pred_file = data_dir+'/dev-v2.0.json'

In [28]:
pred_result_file = output_dir+'dev-v2.0-pred.json'

In [29]:
class Evaluator(keras.callbacks.Callback):
    """
    评估和保存模型
    """
    def __init__(self):
        self.best_val_f1 = 0.

    def on_epoch_end(self, epoch, logs=None):
        metrics = evaluate(data_dir+'/dev-v2.0.json',output_dir+'dev-v2.0-pred.json'
        )
        if float(metrics['f1']) >= self.best_val_f1:
            self.best_val_f1 = float(metrics['f1'])
            model.save_weights(os.path.join(output_dir,'bert_best_model.weights'))
            model.save(os.path.join(output_dir,'bert_best_model.h5'))
        metrics['BEST_F1'] = self.best_val_f1
        print(metrics)

# 13） Access Data

In [30]:
train_generator = data_generator(train_data, batch_size)
evaluator = Evaluator()

# 14) Model Training

In [31]:
epochs=5
model.fit_generator(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    callbacks=[evaluator]
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
125/125 [==============================] - ETA: 0s - loss: 3.9234 - sparse_accuracy: 0.1270

100%|██████████| 500/500 [00:44<00:00, 11.21it/s]


OrderedDict([('exact', 17.8), ('f1', 29.33055888555891), ('total', 500), ('BEST_F1', 29.33055888555891)])
125/125 [==============================] - 173s 1s/step - loss: 3.9234 - sparse_accuracy: 0.1270
Epoch 2/5
125/125 [==============================] - ETA: 0s - loss: 1.9043 - sparse_accuracy: 0.5070

100%|██████████| 500/500 [00:37<00:00, 13.39it/s]


OrderedDict([('exact', 32.6), ('f1', 46.401011607931125), ('total', 500), ('BEST_F1', 46.401011607931125)])
125/125 [==============================] - 144s 1s/step - loss: 1.9043 - sparse_accuracy: 0.5070
Epoch 3/5
125/125 [==============================] - ETA: 0s - loss: 0.9575 - sparse_accuracy: 0.7260

...

OrderedDict([('exact', 37.0), ('f1', 50.3438655788656), ('total', 500), ('BEST_F1', 50.3438655788656)])
125/125 [==============================] - 143s 1s/step - loss: 0.9575 - sparse_accuracy: 0.7260
Epoch 4/5
125/125 [==============================] - ETA: 0s - loss: 0.5071 - sparse_accuracy: 0.8500

100%|██████████| 500/500 [00:43<00:00, 11.38it/s]


OrderedDict([('exact', 35.6), ('f1', 50.31433204050852), ('total', 500), ('BEST_F1', 50.3438655788656)])
125/125 [==============================] - 107s 856ms/step - loss: 0.5071 - sparse_accuracy: 0.8500
Epoch 5/5
125/125 [==============================] - ETA: 0s - loss: 0.3478 - sparse_accuracy: 0.9040

100%|██████████| 500/500 [00:37<00:00, 13.24it/s]


OrderedDict([('exact', 38.4), ('f1', 52.637186963363455), ('total', 500), ('BEST_F1', 52.637186963363455)])
125/125 [==============================] - 138s 1s/step - loss: 0.3478 - sparse_accuracy: 0.9040


# 15) Load The Best Model

In [32]:
from keras.models import load_model
model=load_model(os.path.join(output_dir,'bert_best_model.h5'),custom_objects={'MaskedSoftmax':MaskedSoftmax,'sparse_accuracy':sparse_accuracy})

# 16） Sample Data Testing: Articel Question Answering 

In [34]:
test_data = dev_data[8]

In [35]:
context = test_data[1]

In [37]:
question = test_data[2]

In [40]:
real_answer = test_data[3]

In [45]:
pred_answer = extract_answer(question,context)

In [46]:
context

'Described as being "sexy, seductive and provocative" when performing on stage, Beyoncé has said that she originally created the alter ego "Sasha Fierce" to keep that stage persona separate from who she really is. She described Sasha as being "too aggressive, too strong, too sassy [and] too sexy", stating, "I\'m not like her in real life at all." Sasha was conceived during the making of "Crazy in Love", and Beyoncé introduced her with the release of her 2008 album I Am... Sasha Fierce. In February 2010, she announced in an interview with Allure magazine that she was comfortable enough with herself to no longer need Sasha Fierce. However, Beyoncé announced in May 2012 that she would bring her back for her Revel Presents: Beyoncé Live shows later that month.'

In [47]:
question

'Beyonce brought back Sasha Fierce during which event?'

In [48]:
real_answer

['Revel Presents: Beyoncé Live']

In [49]:
pred_answer

'Revel Presents'